# Post Purchases
Massages a google sheet to use input validation to enter purchases in a batch table format  
Collects results from the sheet and posts them back to Grocy  
Clears the sheet and stores the results in an extra tab

In [ ]:
import common
import json
import pandas as pd
logging = common.start_logger("PostPurcahsesBatch",'DEBUG')
creds = common.GoogleOAuth()
common.GrConf("PostPurchasesBatch")
sheetname = 'products_add'
sheetId = common.CONF.sheets['BatchTx']['sheets']['products_add']
spreadsheetId = common.CONF.sheets['BatchTx']['spreadsheetId']
manualVerify = True #switch for debugging/checking certain results before post

In [ ]:
#Cache master data details based on the headers
target = common.CONF.interface['Tx']['products']['txType']['add']
alias_dict,md_cache,headerdef = common.CONF.cache_header_values(target)
# cache UOM
iterCaller = common.GrGetAPI('Objects','quantity_units')
iterCaller.buildParam("active=1")
iterCaller.get()
mdresult = iterCaller.r.json()
idresult = {}
for result in mdresult:
    index = result.pop('id')
    idresult[index]=result
md_cache['quantity_units'] = idresult

#prepare columns with input validation
#pull the sheet to the front
common.refresh_activate_sheet(creds,spreadsheetId,sheetId,alias_dict,headerdef,logger=logging)

# Read and Validate
Fill in the google sheet with what should be purchased  
When ready, run the next cell to validate it and spit it out into the LastPost sheet for review

In [ ]:
# Take values from Google sheet and validate

start = 'A2'
end = common.rowcol_to_a1(common.CONF.CONF['LocalUser']['Max Rows to Format'],len(headerdef.keys()))
contents = common.spreadsheet_get(creds,spreadsheetId,f'{sheetname}!{start}:{end}')
#Google API will stop returning if the rest of a line is empty ex. if the line is not ready
#Add padding for consistent dataframe and handling of especially not-ready
for result in contents['values']:
    padding = 6-len(result)
    while padding > 0:
        result.append("")
        padding -= 1
dfCol = contents['values'].pop(0)
dfContents = pd.DataFrame(data=contents['values'],columns=dfCol)
#move product name to its own column as reference
dfContents['product'] = dfContents['product_id']
#the hard conversion: spit out product_ids for product strings.
#sure hope I dont reuse strings
prIDlookup = {value: str(key) for key, value in alias_dict['product_id'].items()}
dfContents['product_id'] = dfContents['product_id'].replace(prIDlookup)
dfContents['product_id'] = pd.to_numeric(dfContents['product_id'],errors = 'coerce').astype('Int64')
#easy conversion: spit out location_ids for location strings
#only a few dozen of these so easy to manage as a human
dfContents['location'] = dfContents['location_id']
locIDlookup = {value: str(key) for key, value in alias_dict['location_id'].items()}
dfContents['location_id'] = dfContents['location_id'].replace(locIDlookup)
dfContents['location_id'] = pd.to_numeric(dfContents['location_id'],errors = 'coerce').astype('Int64')
#initialize caution and error and log columns
dfContents['caution'] = False
dfContents['error'] = False
dfContents['log'] = ''
#Run validations
dfContents = dfContents.apply(common.validate_purchase,axis = 1,md_cache=md_cache)
#Convert sales UOM to storage UOM
dfContents = dfContents.apply(common.gr_sales_unit_convert,axis = 1,product_def=md_cache)
dfContents = dfContents.fillna('')
LastPost = dfContents.values.tolist()
LastPost = common.table_csv_extend(LastPost)
LastPost.insert(0,list(dfContents.columns))
#clear the last validation/run
clearLastPost = common.sheets_clear(creds,spreadsheetId,'LastPost','A1:AA1000')
#add the results to the validation tab
common.sheets_append(creds,spreadsheetId,'LastPost',LastPost,'A1')
#optimize columns
updater = common.GoBatchUpdate(creds,spreadsheetId)
optistyle = common.GrStyleGuide('request_styles','optimize_columns')
request = optistyle.guide
request['autoResizeDimensions']['dimensions']['sheetId'] = common.CONF.sheets['BatchTx']['sheets']['LastPost']
request['autoResizeDimensions']['dimensions']['endIndex'] = 100
updater.addReq(request)
#add request to bring the sheet forward
request = {"updateSheetProperties": {
        "properties": {
            "sheetId": common.CONF.sheets['BatchTx']['sheets']['LastPost'],
            "index":0
        },
        "fields": "index"
}
          }
updater.addReq(request)
updater.call()

In [ ]:
#Make a grocy update for good results
dfPosting = dfContents.loc[~dfContents['error']]
dfPosting.loc[:,"best_before_date"] = dfPosting['best_before_date'].apply(pd.to_datetime,format='mixed')
finalResult = []
for row in dfPosting.itertuples():
    purchase = common.GrPostAPI('Tx','products')
    
    request = {
        'amount':int(row.amount),
    }
    if pd.notna(row.best_before_date):
        row.best_before_date
        formatdate = row.best_before_date.strftime('%Y-%m-%d')
        request['best_before_date'] = formatdate
    if row.location_id != '':
        request['location_id'] = row.location_id
    if row.price != "":
        request['price'] = row.price
    purchase.buildTx(request,row.product_id,'add')
    purchase.post()
    result = row._asdict()
    result['log'] = f'{purchase.r},' + result['log']
    finalResult.append(result)
dfResults = pd.DataFrame(finalResult)
dfResults = dfResults.drop(columns="Index")
dfResults['best_before_date'] = dfResults['best_before_date'].dt.strftime('%m/%d/%Y')
# Append the final results below the reviewed data
dfResults = dfResults.fillna('null').replace('null',"")
LastPost = dfResults.values.tolist()
LastPost = common.table_csv_extend(LastPost)
LastPost.insert(0,list(dfResults.columns))
LastPost.insert(0,["Posting Results"])
#add the results to the validation tab
common.sheets_append(creds,spreadsheetId,'LastPost',LastPost,'A1')
#optimize columns
updater = common.GoBatchUpdate(creds,spreadsheetId)
optistyle = common.GrStyleGuide('request_styles','optimize_columns')
request = optistyle.guide
request['autoResizeDimensions']['dimensions']['sheetId'] = common.CONF.sheets['BatchTx']['sheets']['LastPost']
request['autoResizeDimensions']['dimensions']['endIndex'] = 100
updater.addReq(request)
updater.call()

# Try again: collect all errors records and return them to the input sheet to try again
clearLastInput = common.sheets_clear(creds,spreadsheetId,'products_add','A3:AA1000')
errorAttempts = dfContents.loc[dfContents['error']]
additionalErrors = dfResults.loc[~dfResults['log'].str.contains('Response [200]',regex=False)]
#case when to combine results
entryUpdate = []
if len(errorAttempts) > 0 and len(additionalErrors) > 0:
    entryUpdate = pd.concat([errorAttempts,additionalErrors])
elif len(errorAttempts) > 0: entryUpdate = errorAttempts
elif len(additionalErrors) > 0: entryUpdate = additionalErrors
if len(entryUpdate) > 0:
    #proceed with updating the list
    #put aliases back in place
    entryUpdate.loc[:,'product_id'] = entryUpdate['product']
    entryUpdate.loc[:,'location_id'] = entryUpdate['location']
    #drop the columns we dont have in the sheet
    entryUpdate = entryUpdate.drop(columns=['product','location','caution','error','log'])
    LastPost = entryUpdate.values.tolist()
    #add the results to the entry tab
    common.sheets_append(creds,spreadsheetId,'products_add',LastPost,'A3')

In [ ]:
#clear utility, will eventually be used for clearing when we're done
clearLastPost = common.sheets_clear(creds,spreadsheetId,'LastPost','A1:AA1000')